# Ethereum Yellow Paper超要約

Yellow Paperを要約しつつ重要なことを書き足す。

## 2 ブロックチェーンの定式化

Yellow Paperのセクション２では、すべての脱中央集権的ブロックチェーンに共通の構造を定式化している。
ブロックチェーンとは何かしらの内部状態を保持するデータベースであり、トランザクションとブロックの二重構造により内部状態（ステート）を変化させる。

- トランザクションによるステートの変更 - トランザクション$T$が$\Upsilon$関数を用いてステート$\sigma_t$を更新する。
$$ \sigma_{t+1} \equiv \Upsilon(\sigma_t, T)$$

- ブロックによるステートの変更 - ブロック$B$が$\Pi$関数を用いてステート$\sigma_t$を更新する。$B$はトランザクションの列とその他ブロック固有のパラメータで構成されている。$\Pi$はステートをブロック内の全トランザクションの分更新させたのち、マイニング報酬を与えるなどの最終処理関数$\Omega$を用いてステートを更新する。
$$ \sigma_{t+1} \equiv \Pi(\sigma_t, B) $$
$$ B \equiv (\dots, (T0, T1, \dots), \dots) $$
$$ \Pi(\sigma, B) \equiv \Omega(B, \dots\Upsilon(\Upsilon(\sigma, T_0), T_1)\dots) $$

Ethereumにおけるこれらのステート更新関数の具体的な定義はまた後ほど紹介する。

## 2.1 単位
続いてセクション２では通過単位を定義している。最小単位はWeiであり、それを元に次の表のように倍率を設定している。

|Multiplier|Name   |
|----------|-------|
|$10^0$      |Wei    |
|$10^{12}$     |Szabo  |
|$10^{15}$     |Finney |
|$10^{18}$     |Ether  |



In [15]:
class Blockchain:
    def __init__(self, init_state, tx_update, block_finalize):
        # input: state and tx
        # output: state
        self.tx_update = tx_update
        
        # input: block and state
        # output: state
        self.block_finalize = block_finalize
        
        # input: state and block
        # output: state
        self.block_update = lambda s, b: block_finalize(b, reduce(tx_update, b["transactions"], s))
        
        self.state = init_state
    
    def apply_tx(tx):
        self.state = self.tx_update(self.state)
    
    def apply_block(tx):
        self.state = self.block_update(self.state)

wei = 1
szabo = 10**12
finney = 10**15
ether = 10**18

## 4.1 ステートの定義

ここではEthereumにおけるステートの定義を話す。

### 構成

Ethereumにおけるブロックチェーン全体のステート(ワールドステート)とは、アドレスとアカウントステートとの間の写像のこと。つまり、あるアドレスが与えられると、そのアドレスの状態（アカウントステート）を取得できるテーブルのことである。ここでアドレスは160ビットの整数であり、アカウントステートはRLPでエンコードされたデータである。

EthereumではハッシュにKeccak-256関数($\texttt{KEC}$と表記)が用いられる。

world stateはtrieという独自のハッシュ木で構成される。trieはハッシュ化することができ、keyとvalueの組$\mathfrak{I} = \{(k_0, v_0), (k_1, v_1), \dots\}$に対するハッシュを$\texttt{TRIE}(\mathfrak{I})$と表す。

$a$をアカウントとした時のアカウントステート$\sigma[a]$は以下の4つで構成される。
- nonce: アカウントが今までに発行したトランザクション数。$\sigma[a]_n$と表記。
- balance: アカウントの残高。$\sigma[a]_b$と表記。
- storageRoot: アカウントの持つストレージのハッシュ。ストレージは256ビットの値同士の組を保持することができるもので、World Stateと同様にtrieである。$\sigma[a]_s$と表記。
- codeHash: アカウントの持つプログラムコードのハッシュ。$\sigma[a]_c$と表記。コードそのものを$b$と表記することがあり、その時は$\sigma[a]_c = \texttt{KEC}(b)$である。

### データのシリアライズ・ハッシュ化

以下のように、$L_I^*$を使って$\sigma[a]_s$をtrieから取り出すことができる?（意味不明）
$$ \texttt{TRIE}(L_I^*(\sigma[a]_s)) \equiv \sigma[a]_s$$

$\sigma[a]_c = \texttt{KEC}(())$の時、すなわちアカウントのプログラムが空であるとき、そのアカウントは「普通の」アカウント、コントラクトでないアカウントである。

ワールドステートの簡約関数$L_S$を定義する。これはworld stateから存在するアカウントのみを抽出し、エンコードする関数である。
$$L_S(\sigma) \equiv \{p(a) : σ[a] \neq \emptyset \}$$
ここで、
$$p(a) \equiv (\texttt{KEC}(a), \texttt{RLP}((\sigma[a]_n, \sigma[a]_b, \sigma[a]_s, \sigma[a]_c)))$$
である。

$L_S$と$\texttt{TRIE}$を用いてworld stateをハッシュする。以降シリアライズなどのため、world stateは次の形式に沿っているものとする。

1. world stateに含まれる全てのアカウントは存在しないか、もしくはアドレス(trieのキー)が20バイトでそのアカウントステートが正しい形式かである。
$$\forall a : \sigma[a] = \emptyset \lor (a \in \mathbb{B}_{20} \land v(\sigma[a])$$
2. アカウントステートが正しい形式であるとは、nonceが256ビット整数、balanceが256ビット整数、storageRootが32バイト列、codeHashが32バイト列であることである。（256ビット整数と32バイト列は本質的に同じなので区別は趣味の問題である。）
$$v(x) \equiv x_n \in \mathbb{N}_{256} \land x_b \in \mathbb{N}_{256} \land x_s \in \mathbb{B}_{32} \land x_c \in \mathbb{B}_{32}$$

### アカウントの空・死

アカウント$a$が空であるとは、そのコードが空であり、そのnonceが0であり、その残高が0であることである。
$$\texttt{EMPTY}(\sigma,a) \equiv \sigma[a]_c = \texttt{KEC}(()) \land \sigma[a]_n = 0 \land \sigma[a]_b = 0$$

アカウント$a$が死んでいるとは、そのアカウントが存在しないか、空であることである。
$$\texttt{DEAD}(\sigma,a) \equiv \sigma[a] = \emptyset \lor \texttt{EMPTY}(\sigma,a)$$



## 4.2 トランザクションの定義

ここではトランザクションの定義を話す。

### 構成

トランザクションとはEthereumの外部から投稿される、署名された（ステートを更新させる）命令のことである。これは以下の要素からなる。

- nonce : トランザクションを送信したアカウントが今までに発行したトランザクション数。$T_n$と表記。
- gasPrice : 1gasあたりにかかるweiの量を指定する。$T_p$と表記。
- gasLimit : 計算に最大で何gas使えるかを指定する。この値は前払いであり、計算が行われる前から引き落とされる。$T_g$と表記。
- to : 送金先アドレス。160ビット数値。$T_t$と表記。
- value : アドレスtoに送金されるWeiの量。$T_v$と表記。
- v, r, s : トランザクションのECDSA署名。これを用いてトランザクションを作成したアカウント(sender)を計算する。$T_w, T_r, T_s$と表記。

コントラクト作成トランザクションの場合、以下の要素がある。

- init : コントラクトが作成されるときに使われるEVMコードを指定する。可変長バイト列。$T_i$と表記。

initを実行すると、bodyと呼ばれるコードが返される。bodyはコントラクトを呼ぶたびに実行されるコードで、コントラクトへの入力を処理する。つまり、initは「bodyというEVMプログラムを生成するEVMプログラム」ということになる。

コントラクトを呼び出すトランザクションの場合、以下の要素がある。

- data : コントラクトに入力するデータを指定する。可変長バイト列。$T_d$と表記。

### データの処理

実際にトランザクションを処理するために必要な関数を定義していく。

$S(T)$はトランザクションからそのトランザクションを作成したアカウントを計算する。$T_w, T_r, T_s$を使うと、ECDSAの性質から公開鍵を算出できる。

$L_T(T)$は$T_t$の有無からそのトランザクションがコントラクト呼び出し命令なのか作成命令なのかを判別し、適切な配列に並べる関数である。

$$
L_T(T)
\equiv
\left
\{
\begin{array}{l}
(T_n,T_p,T_g,T_t,T_v,T_\mathbf{i},T_w,T_r,T_s) & \text{if} \quad T_t = \emptyset \\
(T_n,T_p,T_g,T_t,T_v,T_\mathbf{d},T_w,T_r,T_s) & \text{otherwise}
\end{array}
\right.
$$

以降、

- nonce, gasPrice, gasLimit, value, r, sは256ビット数値
- vは5ビット数値
- data, initは可変長バイト列
- toは20バイト列　もしくは　空文字列

とする。toを空文字列にした場合、トランザクションはコントラクト作成命令になる。

$$
\begin{equation}
\begin{array}[t]{lclclc}
T_{\mathrm{n}} \in \mathbb{N}_{256} & \wedge & T_{\mathrm{v}} \in \mathbb{N}_{256} & \wedge & T_{\mathrm{p}} \in \mathbb{N}_{256} & \wedge \\
T_{\mathrm{g}} \in \mathbb{N}_{256} & \wedge & T_{\mathrm{w}} \in \mathbb{N}_5 & \wedge & T_{\mathrm{r}} \in \mathbb{N}_{256} & \wedge \\
T_{\mathrm{s}} \in \mathbb{N}_{256} & \wedge & T_{\mathbf{d}} \in \mathbb{B} & \wedge & T_{\mathbf{i}} \in \mathbb{B}
\end{array}
\end{equation}
$$

$$
\begin{equation}
T_{\mathbf{t}} \in \begin{cases} \mathbb{B}_{20} & \text{if} \quad T_{\mathrm{t}} \neq \varnothing \\
\mathbb{B}_{0} & \text{otherwise}\end{cases}
\end{equation}
$$

## 4.3 ブロックの定義

ここではブロックの定義について話す。

### 構成

Ethereumではブロックタイムを短縮するため、一本鎖に繋がれたブロック以外にも報酬が行き渡るようになっている。つまり、ある程度はフォークのブロックにもマイニング報酬が支払われるということだ。

ブロックは以下で構成される。

parentHash: 親のブロックのハッシュ。
ommersHash: 自分のブロックとおなじ祖父を父に持つ、他のブロックを総合したハッシュ。
beneficiary: 
stateRoot: 
transactionsRoot: 
receiptsRoot: 
logsBloom: 
difficulty: 
number: 
gasLimit: 
gasUsed: 
timestamp: 
extraData: 
mixHash: 